# Helmholtz energy (Stixrude - Debye integral)
Required system packages and initialization

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import hashlib
import time
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../endmembers'
try:
    os.mkdir(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference= 'Thermocodegen-v0.6/share/thermocodegen/examples/Systems/MgFeSi2O4_Stixrude/notebooks/Generate_stixrude_endmembers' 

list and get available external functions


In [ ]:
coder.list_external_functions()

In [ ]:
external_funcs = coder.get_external_functions()
db = external_funcs['Debye']

## Declare *T* and *V* to be independent variables
This specifies that the model expression will be defined in terms of the Helmholtz free energy.  Note, that the defauilt model type, *TP*, decalres that the model expression will be the Gibbs free energy.

In [ ]:
model = coder.StdStateModel.from_type('TV')

In [ ]:
T = model.get_symbol_for_t()
V = model.get_symbol_for_v()
Tr = model.get_symbol_for_tr()
Vr = model.get_symbol_for_vr()

### Extract variables and Data from the Stixrude Endmember database

In [ ]:
df = pd.read_csv('data/stixrude_endmembers.csv')
df.columns

In [ ]:
df = df.drop(columns='abbrev')
df

### Generate sympy symbols for parameters in the Stixrude standard state model

In [ ]:
param_strings = df.columns[2:-2]
param_strings = param_strings.insert(len(param_strings),'R')
param_strings

In [ ]:
symbols = [ sym.Symbol(s,real=True) for s in param_strings ]

###  make available to the local dictionary

In [ ]:
s_dict = dict(zip(param_strings,symbols))
locals().update(s_dict)
s_dict

#### Add units and generate parameter tuples needed for coder model

In [ ]:
units = [ 'J/m','', 'J/bar-m','bar','', 'K', '','','J/K-m' ,'J/K-m' ]

In [ ]:
params = list(zip(param_strings,units,symbols))
params

#### Construct a Model of the  Helmholtz free energy
Declare parameters of the Stixrude standard state model:

Define, the isotropic strain contributions to the Helmholtz free energy

In [ ]:
f = ((v0/V)**(sym.S(2)/sym.S(3)) - 1)/2
c1 = sym.S(9)/sym.S(2)*k00*v0
c2 = k0p-4
f

In [ ]:
A = a0 + c1 * f**2 * (1 + c2*f)
A

Define the Debye temperature:

In [ ]:
c3 = 6 * gamma0
c4 = c3 * (-2 + c3 - 3*q)/2

In [ ]:
d0 = theta0 * sym.sqrt( 1 + c3*f + c4 * f**2) 
d0 

try to calculate the actual debye integral in sympy just for fun


In [ ]:
t = sym.symbols('t')
f = sym.log(1 - sym.exp(-t))
I = sym.integrate(f*t**2,t)
(I/t**3).simplify()

Define the Debye Helmholtz free energy ...  
db(x) returns the Debye *integral* 

$$
    D_3(x) = \frac{3}{x^3}\int_0^x \frac{t^3}{e^t - 1} dt
$$

check that debye is actually $D_3$ and returns correct derivative

In [ ]:
db(t).diff(t,t)

In [ ]:
x = d0/T
A_db = n*R*T*(3*sym.log(1-sym.exp(-x)) - db(x))
A_db

... and from that the quasiharmonic approximation to the Helmholtz energy ...

In [ ]:
A_quasi = A_db - A_db.subs(T,Tr)
A_quasi

... and finally the Stixrude model expression for the Helmholtz free energy:

In [ ]:
A +=  A_quasi
A

... and add this expression to the model

In [ ]:
model.add_potential_to_model('A',A, params)

## Save to spud and code-print the model

get and set, model values

In [ ]:
values_dict = model.get_values()
values_dict

In [ ]:
for i, row in df.iterrows():
    values_dict = row.to_dict()
    # clean up names to make them coder compliant
    values_dict['name'] ='{}_stixrude'.format(values_dict['name'])
    values_dict['name'] = values_dict['name'].replace('-','_')
    values_dict['R'] = 8.314472
    print(values_dict['name'])
    values_dict['reference']=reference
    #print(values_dict)
    model.set_values(values_dict)
    model.to_xml(path=SPUD_DIR)